In [15]:
# 필용한 라이브러리를 넣는 과정
import pandas as pd
import numpy as np
import torch
import torch.optim as optim

In [16]:
# GPU 사용 정의 + 결과값을 동결하기 위해 seed를 고정해주는 과정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [17]:
# 데이터셋을 불러오는 과정
data = pd.read_csv('mnist_train.csv')
data2 = pd.read_csv('mnist_test.csv')
submit = pd.read_csv('submission.csv')

In [18]:
# 학습데이터의 x값과 y값을 분류하고 테스트 데이터의 x값을 정의한다.
xy = data.drop(['Unnamed: 0'],axis=1)
xx = data2.drop(['Unnamed: 0'],axis=1)

xy = np.array(xy)
xx = np.array(xx)

train_X = torch.FloatTensor(xy[:,:-1])
train_Y = torch.LongTensor(xy[:,-1])        # LongTensor임을 유의!(Classfication의 경우 결과값이 int형이므로)
test_X = torch.FloatTensor(xx[:,:])

In [19]:
# model의 layer를 정의하는 과정
# layer의 크기가 클수로, 갯수가 많을수록 학습률이 좋지만 과적합의 위험이 있음을 유의!
linear1 = torch.nn.Linear(28*28,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,10,bias=True)      # 분류 문제이고 분류 할 숫자가 0~10까지 있으므로 10으로 마지막 값을 넣는다.
relu = torch.nn.ReLU()                           # 값이 0보다 작으면 0을, 0보다 크면 그대로 값을 출력시켜주는 ReLU 함수 설정
dropout = torch.nn.Dropout(p=0.3)                # 과적합을 예방하기 위한 dropout layer를 정의 지금의 경우 30% dropout

In [20]:
# layer의 가중치 초기화 과정(xavier를 사용했습니다. orthogonal등의 다른 방법 또한 존재합니다.)
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-0.0180, -0.0595,  0.0064,  ..., -0.0023, -0.0860,  0.0191],
        [-0.0545,  0.0292, -0.0036,  ..., -0.0279,  0.1061, -0.0536],
        [-0.0312, -0.0075,  0.0382,  ..., -0.0651,  0.0227, -0.1196],
        ...,
        [ 0.0175,  0.0823, -0.0075,  ..., -0.0867,  0.0492,  0.0200],
        [-0.0306, -0.0145, -0.0415,  ...,  0.0258,  0.1316, -0.0382],
        [ 0.0292,  0.0643, -0.1190,  ...,  0.0634, -0.1017,  0.0353]],
       requires_grad=True)

In [21]:
# 모델의 레이어를 쌓는 과정
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [22]:
# cost를 찾기위한 optimizer를 설정(성능이 가장 좋은 Adam을 사용했습니다.)
loss = torch.nn.CrossEntropyLoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 

In [23]:
# GPU를 사용할 경우 모든 데이터가 같은 device에 있어야 하므로 cpu -> gpu로 옮기는 과정
train_X=train_X.to(device)
train_Y=train_Y.to(device)

# 모델 학습을 진행하는 과정
for epoch in range(101):
    
    optimizer.zero_grad()
    hypothesis = model(train_X)
    cost = loss(hypothesis, train_Y)
    cost.backward()
    optimizer.step()
    c = torch.argmax(hypothesis,axis=1)
    acc = sum(c==train_Y)/len(train_Y)

    if epoch % 10 == 0:
        print("Epoch: {:4d} Cost: {:.7f} Accuraccy: {:.7f}".format(epoch, cost, acc))

Epoch:    0 Cost: 92.7897263 Accuraccy: 0.1014833
Epoch:   10 Cost: 4.5375981 Accuraccy: 0.4686167
Epoch:   20 Cost: 1.5635155 Accuraccy: 0.6366000
Epoch:   30 Cost: 1.0085610 Accuraccy: 0.7105667
Epoch:   40 Cost: 0.7982479 Accuraccy: 0.7642667
Epoch:   50 Cost: 0.6785794 Accuraccy: 0.8029833
Epoch:   60 Cost: 0.6043802 Accuraccy: 0.8283000
Epoch:   70 Cost: 0.5404651 Accuraccy: 0.8472500
Epoch:   80 Cost: 0.5005941 Accuraccy: 0.8586833
Epoch:   90 Cost: 0.4575294 Accuraccy: 0.8714667
Epoch:  100 Cost: 0.4338090 Accuraccy: 0.8791000


In [24]:
# 테스트 데이터를 모델에 넣어 결과값을 계산
test_X = test_X.to(device)
rslt = model(test_X)

# argmax를 사용해 0~10값 중 하나로 분류하는 과정
prdt = torch.argmax(rslt,dim=1)

In [25]:
# 테스트 결과값을 csv파일로 내보내는 과정
prdt = prdt.cpu().detach().numpy()
submit['Label'] = prdt
submit.to_csv('result.csv',index=False)

In [26]:
# 결과 확인
result_chk = pd.read_csv('result.csv')
result_chk

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5
